<a href="https://colab.research.google.com/github/Ilvecho/Project_4/blob/main/P4_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/images

In [ ]:
DATADIR = '/content/gdrive/Ny Drive/images'
CATEGORIES = ['flip', 'notflip']

for use in ['training', 'testing']:
  for category in CATEGORIES:
    path = os.path.join(DATADIR,use,category)  # create path to the data
    for img in os.listdir(path):  # iterate over each image
      img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_RGB)  # convert to array
      plt.imshow(img_array, cmap='rgb')  # graph it
      plt.show()  # display!

      break  # we just want one for now so break
    break  #...and one more!
  break